In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np

class CIFAR10Model(nn.Module):
    def __init__(self, baseMapNum=32, num_classes=10, weight_decay=1e-4):
        super(CIFAR10Model, self).__init__()
        self.conv1 = nn.Conv2d(3, baseMapNum, 3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(baseMapNum)
        self.conv2 = nn.Conv2d(baseMapNum, baseMapNum, 3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(baseMapNum)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout(0.2)

        self.conv3 = nn.Conv2d(baseMapNum, 2*baseMapNum, 3, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(2*baseMapNum)
        self.conv4 = nn.Conv2d(2*baseMapNum, 2*baseMapNum, 3, padding=1, bias=False)
        self.bn4 = nn.BatchNorm2d(2*baseMapNum)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.dropout2 = nn.Dropout(0.3)

        self.conv5 = nn.Conv2d(2*baseMapNum, 4*baseMapNum, 3, padding=1, bias=False)
        self.bn5 = nn.BatchNorm2d(4*baseMapNum)
        self.conv6 = nn.Conv2d(4*baseMapNum, 4*baseMapNum, 3, padding=1, bias=False)
        self.bn6 = nn.BatchNorm2d(4*baseMapNum)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.dropout3 = nn.Dropout(0.4)

        self.flatten = nn.Flatten()
        self.fc = nn.Linear(4*baseMapNum*4*4, num_classes)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool1(x)
        x = self.dropout1(x)

        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool2(x)
        x = self.dropout2(x)

        x = F.relu(self.bn5(self.conv5(x)))
        x = F.relu(self.bn6(self.conv6(x)))
        x = self.pool3(x)
        x = self.dropout3(x)

        x = self.flatten(x)
        x = self.fc(x)
        return x


In [14]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010]),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4)
])

batch_size = 128

train_dataset = datasets.CIFAR10(root='./data', train=True, download=False, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.CIFAR10(root='./data', train=False, download=False, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CIFAR10Model().to(device)
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001, weight_decay=1e-6)
criterion = nn.CrossEntropyLoss()
epochs = 150

for epoch in range(epochs):
    model.train()
    total_loss_train = 0
    total_acc_train = 0

    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss_train += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_acc_train += (predicted == labels).sum().item()

    model.eval()
    total_loss_val = 0
    total_acc_val = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            total_loss_val += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_acc_val += (predicted == labels).sum().item()

    print(
        f'''Epochs: {epoch + 1} 
          | Train Loss: {total_loss_train / len(train_loader.dataset): .5f} 
          | Train Accuracy: {total_acc_train / len(train_loader.dataset): .5f} 
          | Val Loss: {total_loss_val / len(test_loader.dataset): .5f} 
          | Val Accuracy: {total_acc_val / len(test_loader.dataset): .5f}'''
    )

# Save model
torch.save(model.state_dict(), 'cifar10_pytorch_model.pth')


Epochs: 1 
          | Train Loss:  0.01449 
          | Train Accuracy:  0.35364 
          | Val Loss:  0.01136 
          | Val Accuracy:  0.46390
Epochs: 2 
          | Train Loss:  0.00992 
          | Train Accuracy:  0.53884 
          | Val Loss:  0.01036 
          | Val Accuracy:  0.54810
Epochs: 3 
          | Train Loss:  0.00827 
          | Train Accuracy:  0.62398 
          | Val Loss:  0.00782 
          | Val Accuracy:  0.65170
Epochs: 4 
          | Train Loss:  0.00730 
          | Train Accuracy:  0.66868 
          | Val Loss:  0.00705 
          | Val Accuracy:  0.68710
Epochs: 5 
          | Train Loss:  0.00665 
          | Train Accuracy:  0.69908 
          | Val Loss:  0.00744 
          | Val Accuracy:  0.66240
Epochs: 6 
          | Train Loss:  0.00616 
          | Train Accuracy:  0.72392 
          | Val Loss:  0.00653 
          | Val Accuracy:  0.70820
Epochs: 7 
          | Train Loss:  0.00581 
          | Train Accuracy:  0.74026 
          | Val L